In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transformers
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm
import sklearn
from sklearn.preprocessing import normalize
from tqdm import tqdm

import glob
import time

import math
import os

In [4]:
initial_states = []

path = "../input_data/"

for dir, sub_dir, files in os.walk(path):
    for file in sorted(files):
        #print(file)
        temp = pd.read_csv((path+file),index_col=None, header=0)
        initial_states.append(temp)

initial_states_df = pd.concat(initial_states,axis=0,ignore_index=True)

hold = initial_states_df
x = initial_states_df.iloc[:,2:].values
x = normalize(x,norm='l2')
hold = pd.concat([hold['File ID'],pd.DataFrame(x)],axis=1)
initial_states_normalized = hold
#'2000-08-02 04:50:33'
timestamps = initial_states_df['Timestamp']
initial_states_df

,File ID,Timestamp,Semi-major Axis (km),Eccentricity,Inclination (deg),RAAN (deg),Argument of Perigee (deg),True Anomaly (deg),Latitude (deg),Longitude (deg),Altitude (km)
0,0,2000-08-02 04:50:33,6826.387247,0.003882,87.275306,144.135111,257.314389,102.383270,43.637815,-62.543128,466.448890
1,1,2000-08-03 19:51:01,6826.327748,0.003879,87.275694,143.529694,250.438806,109.273118,43.444458,70.139709,463.435053
2,2,2000-08-05 05:40:05,6819.634802,0.004114,87.268611,142.972111,244.549389,115.138737,19.764250,104.521278,471.625453
3,3,2000-08-06 05:02:20,6819.606603,0.004134,87.268194,142.608389,241.172000,118.545161,12.450738,112.239558,470.385914
4,4,2000-08-08 20:54:57,6819.425918,0.004178,87.264611,141.605111,228.779611,130.982981,-8.776992,-130.559634,468.911226
...,...,...,...,...,...,...,...,...,...,...,...
8114,8114,2019-12-25 00:00:00,6765.013678,0.005730,87.863978,102.587920,240.608198,187.758342,69.535173,108.291937,443.930606
8115,8115,2019-12-27 00:00:00,6801.130577,0.002172,90.690901,99.760357,152.602156,226.350702,20.225336,99.507926,435.492910
8116,8116,2019-12-28 00:00:00,6805.864837,0.001925,91.053632,96.918243,43.442569,131.160787,4.265831,-83.003782,436.652863
8117,8117,2019-12-30 00:00:00,6774.300973,0.004785,88.598951,92.814340,320.652681,164.621884,53.716936,-89.084995,441.438958


In [5]:
class FullDataset(Dataset):
    def __init__(self, initial_states_df, timestamps, density_length=432, goes_length=86400, omni2_length=1440, density_dir='../data/dataset/test/sat_density', goes_dir="../data/dataset/test/goes",
                 omni2_dir="../data/dataset/test/omni2"):
        self.data = initial_states_df.reset_index(drop=True)
        self.density_dir = density_dir
        self.goes_dir = goes_dir
        self.omni2_dir = omni2_dir
        self.density_length = density_length
        self.goes_length = goes_length
        self.omni2_length = omni2_length
        self.timestamps = timestamps
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        #timestamp = row['Timestamp']
        static_input = torch.tensor(row.drop('File ID').fillna(0.0).values, dtype=torch.float32)
        
        file_id = str(int(row['File ID'])).zfill(5)
        
        density_file = glob.glob(os.path.join(self.density_dir, f"*-{file_id}-*.csv"))
        goes_file = glob.glob(os.path.join(self.goes_dir, f"*-{file_id}-*.csv"))
        omni2_file = glob.glob(os.path.join(self.omni2_dir, f"*-{file_id}-*.csv"))

        #pos = len(self.density_dir)+1
        #density_sat = density_file[0][pos:pos+6]

        density_df = ((pd.read_csv(density_file[0])))
        density_df['Orbit Mean Density (kg/m^3)'] = np.where(density_df['Orbit Mean Density (kg/m^3)']>=1,np.nan,density_df['Orbit Mean Density (kg/m^3)'])
        if density_df.shape[0] > self.density_length:
            density_df = density_df[:self.density_length]
        elif density_df.shape[0] < self.density_length:
            padding = pd.DataFrame(np.empty((self.density_length-density_df.shape[0],2)),columns=density_df.columns)
            padding[:] = np.nan
            density_df = pd.concat((density_df,padding),ignore_index=True)
        density_df_mask = (pd.notnull(density_df)).astype(int)
        density_tensor = torch.tensor(density_df['Orbit Mean Density (kg/m^3)'].fillna(0.0).values, dtype=torch.float32)
        density_df_mask_tensor = torch.tensor(density_df_mask.iloc[:,1].values, dtype=torch.float32)

        goes_df = pd.read_csv(goes_file[0])
        if goes_df.shape[0] > self.goes_length:
            goes_df = goes_df[goes_df.shape[0]-self.goes_length:goes_df.shape[0]]
        elif goes_df.shape[0] < self.goes_length:
            padding = pd.DataFrame(np.empty((self.goes_length-goes_df.shape[0],43)),columns=goes_df.columns)
            padding[:] = np.nan
            goes_df = pd.concat((padding,goes_df),ignore_index=True)
        goes_mask = (~pd.isnull(goes_df)).astype(int)
        goes_valid_mask = ((goes_df['xrsa_flag'] == 0.0) & (goes_df['xrsb_flag'] == 0.0)).astype(int)
        goes_mask = goes_mask.mul(goes_valid_mask.values,axis=0)
        goes_tensor = torch.tensor(normalize(goes_df.iloc[:, 1:].fillna(0.0).values, norm='l2'), dtype=torch.float32)
        goes_mask_tensor = torch.tensor(goes_mask.iloc[:, 1:].values, dtype=torch.float32)
        
        omni2_df = pd.read_csv(omni2_file[0])
        if omni2_df.shape[0] > self.omni2_length:
            omni2_df = omni2_df[omni2_df.shape[0]-self.omni2_length:omni2_df.shape[0]]
        elif goes_df.shape[0] < self.omni2_length:
            padding = pd.DataFrame(np.empty((self.omni2_length-omni2_df.shape[0],58)),columns=omni2_df.columns)
            padding[:] = np.nan
            omni2_df = pd.concat((padding,omni2_df),ignore_index=True)
        omni2_tensor = torch.tensor(normalize(omni2_df.iloc[:, :57].fillna(0.0).values.astype(float), norm='l2'), dtype=torch.float32)
        omni2_mask = (~pd.isnull(omni2_df)).astype(int)
        omni2_mask_tensor = torch.tensor(omni2_mask.iloc[:, :57].values, dtype=torch.float32) 

        return static_input, density_tensor, density_df_mask_tensor, goes_tensor, goes_mask_tensor, omni2_tensor, omni2_mask_tensor, self.timestamps[idx], idx



In [21]:

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=10000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(-torch.arange(0, d_model, 2) * math.log(10000.0) / d_model)
        pe[:, 0::2] = torch.sin(pos * div_term)
        pe[:, 1::2] = torch.cos(pos * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class STORMTransformer(nn.Module):
    def __init__(self,
                 static_dim=9,
                 omni2_dim=57,
                 goes_dim=42,
                 d_model=128,
                 output_len=432,
                 nhead=4,
                 num_layers=2,
                 dropout=0.1):
        super().__init__()

        self.static_encoder = nn.Sequential(
            nn.Linear(static_dim, d_model),
            nn.ReLU(),
            nn.LayerNorm(d_model)
        )

        self.omni2_proj = nn.Linear(omni2_dim, d_model)
        self.goes_proj = nn.Linear(goes_dim, d_model)

        self.omni2_pos = PositionalEncoding(d_model)
        self.goes_pos = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.omni2_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.goes_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fusion = nn.Sequential(
            nn.Linear(d_model * 3, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, output_len)
        )

    def forward(self, static_input, omni2_seq, goes_seq, omni2_mask=None, goes_mask=None):
        B = static_input.size(0)

        static_embed = self.static_encoder(static_input)

        omni2_embed = self.omni2_proj(omni2_seq)
        omni2_embed = self.omni2_pos(omni2_embed)
        omni2_key_mask = (~omni2_mask.bool()).any(dim=-1) if omni2_mask is not None else None
        omni2_out = self.omni2_encoder(omni2_embed, src_key_padding_mask=omni2_key_mask)
        omni2_summary = omni2_out.mean(dim=1)

        if goes_seq.shape[1] > 1024:
            step = goes_seq.shape[1] // 1024
            goes_seq = goes_seq[:, ::step, :]
            goes_mask = goes_mask[:, ::step, :] if goes_mask is not None else None

        goes_embed = self.goes_proj(goes_seq)
        goes_embed = self.goes_pos(goes_embed)
        goes_key_mask = (~goes_mask.bool()).any(dim=-1) if goes_mask is not None else None
        goes_out = self.goes_encoder(goes_embed, src_key_padding_mask=goes_key_mask)
        goes_summary = goes_out.mean(dim=1)

        combined = torch.cat((static_embed, omni2_summary, goes_summary), dim=-1)
        return self.fusion(combined)

def masked_mse_loss(preds, targets, mask, eps=1e-8):
    
    diff = (targets - preds) * mask
    sq = torch.square(diff)
    sum = torch.sum(sq)
    N = torch.sum(mask)
    # print(sum)
    # print(N)
    loss = torch.sqrt((sum/N))
    return loss


def eval_storm_transformer(initial_states_df, timestamps, device=None):

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    val_dataset = FullDataset(initial_states_df, timestamps)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

    model = STORMTransformer().to(device)
    model.load_state_dict(torch.load("../checkpoints/storm_epoch_7.pt", weights_only=False))

    ids_list = []
    timestamps_list = []
    preds_list = []

    counter = 0
    model.train()
    with torch.no_grad():
        for batch in tqdm(val_loader):
            if counter >3:
                break
            counter += 1
            (static_input, 
            density_tensor, 
            density_mask_tensor, 
            goes_tensor, 
            goes_mask_tensor, 
            omni2_tensor, 
            omni2_mask_tensor, 
            timestamps,
            ids) = batch

            static_input = static_input.to(device)
            density_tensor = density_tensor.to(device)
            density_mask_tensor = density_mask_tensor.to(device)
            goes_tensor = goes_tensor.to(device)
            goes_mask_tensor = goes_mask_tensor.to(device)
            omni2_tensor = omni2_tensor.to(device)
            omni2_mask_tensor = omni2_mask_tensor.to(device)

            preds = model(static_input, omni2_tensor, goes_tensor, omni2_mask_tensor, goes_mask_tensor)
            #print (preds)
            loss = masked_mse_loss(preds,density_tensor,density_mask_tensor)

            ids_list = ids_list + ids.tolist()
            timestamps_list = timestamps_list + list(timestamps)
            preds_list = preds_list + (preds.to('cpu')).tolist()
            print (density_tensor.to('cpu').tolist())
            print (preds.to('cpu').tolist())
            print ("loss:",loss)
    # print (ids_list)
    # print (timestamps_list)
    # print (preds_list)
    
    pred_df = pd.DataFrame(preds_list)
    ids_df = pd.DataFrame(ids_list)
    timestamps_df = pd.DataFrame(timestamps_list)
    out = pd.concat([ids_df,timestamps_df, pred_df],axis=1)
    return (out)
            #break

In [7]:
# val_dataset = FullDataset(initial_states_normalized,timestamps)
# val_dataset[1]


In [22]:
temp = eval_storm_transformer(initial_states_normalized,timestamps)

  0%|          | 1/1015 [00:01<22:33,  1.33s/it]

[[1.8035890706272206e-12, 1.8122302703621473e-12, 1.8237961054573537e-12, 1.8277510581421463e-12, 1.7879032672765693e-12, 1.7734410944977852e-12, 1.780860506804538e-12, 1.7866580610814697e-12, 1.7610681793053806e-12, 1.718779090408018e-12, 1.6838584563153014e-12, 1.6832500020561025e-12, 1.6830641698037385e-12, 1.6796386330397706e-12, 1.6685630741669621e-12, 1.6525588406382519e-12, 1.6627112013611889e-12, 1.6450013009747244e-12, 1.6352829463814333e-12, 1.616827657001385e-12, 1.6118507353488076e-12, 1.6164261769369137e-12, 1.6250474273518667e-12, 1.6274373342006765e-12, 1.6291781292088192e-12, 1.6266687432806015e-12, 1.6288314013540584e-12, 1.648534824275072e-12, 1.6632053807114078e-12, 1.6657562915828317e-12, 1.6494386152060558e-12, 1.6542319814308315e-12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.6113075500603924e-12, 1.5881239465859176e-12, 1.585979719949393e-12, 1.5824535692238184e-12, 1.5625522794063262e-12, 1.5509981536945827e-12, 1.5453

  0%|          | 2/1015 [00:02<22:42,  1.34s/it]

[[1.9601679220743184e-12, 1.964685142005762e-12, 1.959295789846771e-12, 1.94431927155736e-12, 1.9117455014872053e-12, 1.922168587492612e-12, 1.9262115773938104e-12, 1.922401907800131e-12, 1.9299585801019203e-12, 1.935402792890839e-12, 1.938447449431613e-12, 1.9543494426954577e-12, 1.9640265976061944e-12, 2.003273632047997e-12, 2.047555484538388e-12, 2.061850039661306e-12, 2.059456554945327e-12, 2.050025730768179e-12, 2.0630608766475378e-12, 2.066350346038859e-12, 2.0611479103344044e-12, 2.0554454405879996e-12, 2.0562551227704118e-12, 2.0075525443419284e-12, 2.035251307763719e-12, 2.0358801450237607e-12, 2.0429823197748442e-12, 2.0320219031727538e-12, 2.044004722423498e-12, 2.0433780535678014e-12, 2.0341710087190545e-12, 2.0273897578110267e-12, 2.0427463973821114e-12, 2.01578098830979e-12, 2.0348178437351594e-12, 2.051478344838875e-12, 2.0489040152005256e-12, 2.0295449348894934e-12, 2.02585756330087e-12, 2.024446365753163e-12, 2.019776273315399e-12, 2.006759775713407e-12, 1.996465276085

  0%|          | 3/1015 [00:04<22:34,  1.34s/it]

[[1.6671633691622834e-12, 1.668400335420872e-12, 1.6752400248259969e-12, 1.6785305784194904e-12, 1.6829363423676025e-12, 1.7001548823294965e-12, 1.7161828598857842e-12, 1.723599670107323e-12, 1.7364038809239424e-12, 1.741075057563879e-12, 1.7447397693270972e-12, 1.7568067226664263e-12, 1.7646771630769331e-12, 1.7622837867811714e-12, 1.7739632462640542e-12, 1.7965920634868682e-12, 1.802191642447104e-12, 1.8070041990503327e-12, 1.8434110574810925e-12, 1.8741258545063033e-12, 1.8676360371422396e-12, 1.8771191198641013e-12, 1.884590357034699e-12, 1.8733595404107906e-12, 1.8494238258892626e-12, 1.8424719215592855e-12, 1.8492655323720797e-12, 1.8407705915102213e-12, 1.8182101874444911e-12, 1.8149013108342826e-12, 1.826191541737243e-12, 1.835779358388967e-12, 1.8440993090201863e-12, 1.8467868293653433e-12, 1.863306384186636e-12, 1.8805408619204655e-12, 1.8799135425434654e-12, 1.879398112830666e-12, 1.908713204851198e-12, 1.9404434656311587e-12, 1.94434290716472e-12, 1.942744142641173e-12, 1.9

  0%|          | 4/1015 [00:05<22:31,  1.34s/it]

[[2.0474654957580718e-12, 2.0556464516707784e-12, 2.0577285535228196e-12, 2.037903916798922e-12, 2.069919539590681e-12, 2.0773508781213312e-12, 2.0694591873482437e-12, 2.0866077962700125e-12, 2.1102989147814277e-12, 2.1287028129785e-12, 2.148610065907941e-12, 2.1559930490216983e-12, 2.166410063494939e-12, 2.1301517407618098e-12, 2.1148219894046028e-12, 2.1173698645099437e-12, 2.133576952265126e-12, 2.0878624350240127e-12, 2.1175708755927225e-12, 2.1400353277661877e-12, 2.1658985369099604e-12, 2.1755065197220924e-12, 2.132874172416921e-12, 2.1289135818808314e-12, 2.1274759298001156e-12, 2.1384382979661165e-12, 2.1510239336247627e-12, 2.1468764266341367e-12, 2.1453329564213863e-12, 2.1425413526676707e-12, 2.1429884776436037e-12, 2.154752288055506e-12, 2.1597402685702427e-12, 2.1619006498191373e-12, 2.1632298816826045e-12, 2.163891895529124e-12, 2.164466739520976e-12, 2.198327457569871e-12, 2.183284369267069e-12, 2.1604176780876116e-12, 2.1659050421229953e-12, 2.1990731718241063e-12, 2.21

  0%|          | 4/1015 [00:06<28:14,  1.68s/it]


In [81]:
temp

,0,0,0,1,2,3,4,5,6,7,...,422,423,424,425,426,427,428,429,430,431
0,0,2000-08-02 04:50:33,1.627300e-12,1.463745e-12,1.776746e-12,1.483979e-12,1.485336e-12,1.594291e-12,1.457895e-12,1.660428e-12,...,1.340833e-12,1.346918e-12,1.780044e-12,1.520346e-12,1.258779e-12,1.299515e-12,1.482196e-12,1.504364e-12,1.262871e-12,1.783802e-12
1,1,2000-08-03 19:51:01,1.627300e-12,1.463745e-12,1.776746e-12,1.483979e-12,1.485336e-12,1.594291e-12,1.457895e-12,1.660428e-12,...,1.340833e-12,1.346918e-12,1.780044e-12,1.520346e-12,1.258779e-12,1.299515e-12,1.482196e-12,1.504364e-12,1.262871e-12,1.783802e-12
2,2,2000-08-05 05:40:05,1.627300e-12,1.463745e-12,1.776746e-12,1.483979e-12,1.485336e-12,1.594291e-12,1.457895e-12,1.660428e-12,...,1.340833e-12,1.346918e-12,1.780044e-12,1.520346e-12,1.258779e-12,1.299515e-12,1.482196e-12,1.504364e-12,1.262871e-12,1.783802e-12
3,3,2000-08-06 05:02:20,1.627300e-12,1.463745e-12,1.776746e-12,1.483979e-12,1.485336e-12,1.594291e-12,1.457895e-12,1.660428e-12,...,1.340833e-12,1.346918e-12,1.780044e-12,1.520346e-12,1.258779e-12,1.299515e-12,1.482196e-12,1.504364e-12,1.262871e-12,1.783802e-12
4,4,2000-08-08 20:54:57,1.627300e-12,1.463745e-12,1.776746e-12,1.483979e-12,1.485336e-12,1.594291e-12,1.457895e-12,1.660428e-12,...,1.340833e-12,1.346918e-12,1.780044e-12,1.520346e-12,1.258779e-12,1.299515e-12,1.482196e-12,1.504364e-12,1.262871e-12,1.783802e-12
5,5,2000-08-09 20:17:05,1.627300e-12,1.463745e-12,1.776746e-12,1.483979e-12,1.485336e-12,1.594291e-12,1.457895e-12,1.660428e-12,...,1.340833e-12,1.346918e-12,1.780044e-12,1.520346e-12,1.258779e-12,1.299515e-12,1.482196e-12,1.504364e-12,1.262871e-12,1.783802e-12
6,6,2000-08-11 19:01:16,1.627300e-12,1.463745e-12,1.776746e-12,1.483979e-12,1.485336e-12,1.594291e-12,1.457895e-12,1.660428e-12,...,1.340833e-12,1.346918e-12,1.780044e-12,1.520346e-12,1.258779e-12,1.299515e-12,1.482196e-12,1.504364e-12,1.262871e-12,1.783802e-12
7,7,2000-09-03 02:50:45,1.627300e-12,1.463745e-12,1.776746e-12,1.483979e-12,1.485336e-12,1.594291e-12,1.457895e-12,1.660428e-12,...,1.340833e-12,1.346918e-12,1.780044e-12,1.520346e-12,1.258779e-12,1.299515e-12,1.482196e-12,1.504364e-12,1.262871e-12,1.783802e-12
8,8,2000-09-05 15:40:48,1.627300e-12,1.463745e-12,1.776746e-12,1.483979e-12,1.485336e-12,1.594291e-12,1.457895e-12,1.660428e-12,...,1.340833e-12,1.346918e-12,1.780044e-12,1.520346e-12,1.258779e-12,1.299515e-12,1.482196e-12,1.504364e-12,1.262871e-12,1.783802e-12
9,9,2000-09-06 02:35:56,1.627300e-12,1.463745e-12,1.776746e-12,1.483979e-12,1.485336e-12,1.594291e-12,1.457895e-12,1.660428e-12,...,1.340833e-12,1.346918e-12,1.780044e-12,1.520346e-12,1.258779e-12,1.299515e-12,1.482196e-12,1.504364e-12,1.262871e-12,1.783802e-12
